In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = "./data"

In [3]:
master = pd.read_csv('master_data_all_factors.csv')

In [4]:
factor_1 = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA']
factor_2 = ['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s']
factor_3 = ['Mkt', 'S-F', 'H-F', 'R-F', 'C-F']
factor_4 = ['Mkt', 'S-F', 'H_s-F', 'R_s-F', 'C_s-F']
factor_5 = ['Mkt', 'S-F', 'L-F', 'W-F', 'A-F']
factor_6 = ['Mkt', 'S-F', 'L_s-F', 'W_s-F', 'A_s-F']

factors = [factor_1, factor_2, factor_3, factor_4, factor_5, factor_6]

In [5]:
np.random.seed(42)

def simulate_sharpe_ratios(returns, factor, n_simulations=100):
    n_months = len(returns)
    n_pairs = n_months // 2
    pairs = [(i, i+1) for i in range(0, n_months, 2)]

    fs_sharpe_ratios = []
    is_sharpe_ratios = []
    os_sharpe_ratios = []

    for _ in range(n_simulations):
        # fs
        fs_sample = returns.sample(n=n_months, replace=True)
        fs_sharpe_ratio = squared_sharpe_ratio(fs_sample, factor)
        fs_sharpe_ratios.append(fs_sharpe_ratio)

        # is and os
        sampled_pairs = np.random.choice(range(n_pairs), size=n_pairs, replace=True)
        is_sample_indices = []
        os_sample_indices = []

        for pair_index in sampled_pairs:
            month1, month2 = pairs[pair_index]
            if np.random.rand() < 0.5:
                is_sample_indices.append(month1)
                os_sample_indices.append(month2)
            else:
                is_sample_indices.append(month2)
                os_sample_indices.append(month1)

        is_sample = returns.iloc[is_sample_indices]
        os_sample = returns.iloc[os_sample_indices]

        is_sharpe_ratio = squared_sharpe_ratio(is_sample, factor)
        is_sharpe_ratios.append(is_sharpe_ratio)
        is_portfolio_weights = tangency_portfolio_weights(is_sample, factor)
        is_portfolio_weights = is_portfolio_weights.ravel()
        
        os_sharpe_ratio = squared_sharpe_ratio(os_sample, factor, is_portfolio_weights)
        os_sharpe_ratios.append(os_sharpe_ratio)


    # Convert to DataFrame
    sharpe_ratios = pd.DataFrame({
        'FS': fs_sharpe_ratios,
        'IS': is_sharpe_ratios,
        'OS': os_sharpe_ratios
    })

    return sharpe_ratios

def squared_sharpe_ratio(sample, factor, weights=None):

    if weights is None:
        mu_f = sample[factor].mean().values.reshape(-1, 1) 
        sigma_f = sample[factor].cov().values
        sigma_f_inv = np.linalg.inv(sigma_f)

        # maximum squared Sharpe ratio
        sh2_f = mu_f.T @ sigma_f_inv @ mu_f
        sh2_f = sh2_f[0,0]

    else:
        returns = (sample[factor] * weights).sum(axis=1)
        mu_f = returns.mean()
        sigma_f = returns.std()

        sh2_f = (mu_f / sigma_f) ** 2

    return sh2_f

def tangency_portfolio_weights(is_sample, factor):
    """
    Calculate the tangency portfolio weights.
    
    :param expected_returns: A vector of expected returns (mu)
    :param covariance_matrix: The covariance matrix (Sigma)
    :param risk_free_rate: The risk-free rate (rf)
    :return: A vector of tangency portfolio weights (t)

    source: https://bookdown.org/compfinezbook/introcompfinr/Efficient-portfolios-of.html - eq. 12.26
    """
    mu_f = is_sample[factor].mean().values.reshape(-1, 1) 
    sigma_f = is_sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    numerator = sigma_f_inv @ (mu_f)
    denominator = np.ones(len(mu_f)) @ numerator
    
    tangency_weights = numerator / denominator
    
    return tangency_weights

In [6]:
summary_df = pd.DataFrame(columns=[
    'Model', 'Actual', 'FS Average', 'FS Median', 
    'IS Average', 'IS Median', 'OS Average', 'OS Median'
])

N = 100000

for factor in factors:
    
    actual_sharpe = squared_sharpe_ratio(master, factor)
    sharpe_ratios = simulate_sharpe_ratios(master, factor, n_simulations=N)
    summary_stats = sharpe_ratios.describe().loc[['mean', '50%']]

    model_name = ', '.join(factor)

    fs_avg = summary_stats.loc['mean', 'FS']
    fs_median = summary_stats.loc['50%', 'FS']
    is_avg = summary_stats.loc['mean', 'IS']
    is_median = summary_stats.loc['50%', 'IS']
    os_avg = summary_stats.loc['mean', 'OS']
    os_median = summary_stats.loc['50%', 'OS']

    model_df = pd.DataFrame({
        'Model': [model_name],
        'Actual': [actual_sharpe],
        'FS Average': [fs_avg],
        'FS Median': [fs_median],
        'IS Average': [is_avg],
        'IS Median': [is_median],
        'OS Average': [os_avg],
        'OS Median': [os_median]
    })

    summary_df = pd.concat([summary_df, model_df], ignore_index=True)

summary_df

,Model,Actual,FS Average,FS Median,IS Average,IS Median,OS Average,OS Median
0,"Mkt, SMB, HML, RMW, CMA",0.107304,0.117650,0.115960,0.128092,0.124481,0.101552,0.098016
1,"Mkt, SMB, HML_s, RMW_s, CMA_s",0.140338,0.150274,0.148656,0.160686,0.157268,0.133586,0.129991
2,"Mkt, S-F, H-F, R-F, C-F",0.088350,0.098460,0.096570,0.108636,0.105078,0.083073,0.079546
3,"Mkt, S-F, H_s-F, R_s-F, C_s-F",0.079124,0.088803,0.087008,0.098514,0.095015,0.073807,0.070287
4,"Mkt, S-F, L-F, W-F, A-F",0.070671,0.079885,0.078020,0.088979,0.085435,0.065131,0.061467
5,"Mkt, S-F, L_s-F, W_s-F, A_s-F",0.078162,0.087022,0.085375,0.096274,0.092427,0.072635,0.069072


In [7]:
summary_df.to_excel('./result/tableA1.xlsx', index=False)

In [6]:
master = pd.read_csv('master_data_all_factors.csv')
table_3_summary = pd.read_excel('./result/tableA1.xlsx')

In [7]:
top_3_actual = table_3_summary.sort_values('Actual', ascending=False).head(3)
top_3_actual

,Model,Actual,FS Average,FS Median,IS Average,IS Median,OS Average,OS Median
1,"Mkt, SMB, HML_s, RMW_s, CMA_s",0.140338,0.150274,0.148656,0.160686,0.157268,0.133586,0.129991
0,"Mkt, SMB, HML, RMW, CMA",0.107304,0.117650,0.115960,0.128092,0.124481,0.101552,0.098016
2,"Mkt, S-F, H-F, R-F, C-F",0.088350,0.098460,0.096570,0.108636,0.105078,0.083073,0.079546


In [8]:
col_3_factors = [model.split(', ') for model in top_3_actual['Model'].values]
col_3_factors

[['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s'],
 ['Mkt', 'SMB', 'HML', 'RMW', 'CMA'],
 ['Mkt', 'S-F', 'H-F', 'R-F', 'C-F']]

In [9]:
np.random.seed(42)

def squared_sharpe_ratio(sample, factor, weights=None):
    if weights is None:
        mu_f = sample[factor].mean().values.reshape(-1, 1)
    else:
        mu_f = (sample[factor] * weights).sum(axis=1)

    sigma_f = sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    # maximum squared Sharpe ratio
    sh2_f = mu_f.T @ sigma_f_inv @ mu_f

    return sh2_f[0, 0]

def simulate_sharpe_ratios(returns, row_factor, col_3_factors, n_simulations=100):
    n_months = len(returns)
    n_pairs = n_months // 2
    pairs = [(i, i+1) for i in range(0, n_months, 2)]

    fs_sharpe_ratios_diff = [[], [], []]
    is_sharpe_ratios_diff = [[], [], []]
    os_sharpe_ratios_diff = [[], [], []]

    for _ in range(n_simulations):
        # fs
        fs_sample_row = returns.sample(n=n_months, replace=True)
        fs_sharpe_ratio_row = squared_sharpe_ratio(fs_sample_row, row_factor)

        for i, col_factor in enumerate(col_3_factors):
            if row_factor == col_factor:
                continue
            # fs col
            fs_sample_col = returns.sample(n=n_months, replace=True)
            fs_sharpe_ratio_col = squared_sharpe_ratio(fs_sample_col, col_factor)

            fs_sharpe_ratios_diff[i].append(fs_sharpe_ratio_col - fs_sharpe_ratio_row)

            # is and os
            sampled_pairs = np.random.choice(range(n_pairs), size=n_pairs, replace=True)
            is_sample_indices = []
            os_sample_indices = []

            for pair_index in sampled_pairs:
                month1, month2 = pairs[pair_index]
                if np.random.rand() < 0.5:
                    is_sample_indices.append(month1)
                    os_sample_indices.append(month2)
                else:
                    is_sample_indices.append(month2)
                    os_sample_indices.append(month1)

            is_sample_row = returns.iloc[is_sample_indices]
            os_sample_row = returns.iloc[os_sample_indices]

            is_sample_col = returns.iloc[is_sample_indices]
            os_sample_col = returns.iloc[os_sample_indices]

            is_sharpe_ratio_row = squared_sharpe_ratio(is_sample_row, row_factor)
            is_sharpe_ratio_col = squared_sharpe_ratio(is_sample_col, col_factor)
            is_sharpe_ratios_diff[i].append(is_sharpe_ratio_col - is_sharpe_ratio_row)

            is_weights_row = tangency_portfolio_weights(is_sample_row, row_factor)
            os_sharpe_ratio_row = squared_sharpe_ratio(os_sample_row, row_factor, is_weights_row)

            is_weights_col = tangency_portfolio_weights(is_sample_col, col_factor)
            os_sharpe_ratio_col = squared_sharpe_ratio(os_sample_col, col_factor, is_weights_col)
            os_sharpe_ratios_diff[i].append(os_sharpe_ratio_col - os_sharpe_ratio_row)

    stats = {'Model': [], 'FS Avg Diff': [], 'FS Med Diff': [], 'FS % < 0': [],
             'IS Avg Diff': [], 'IS Med Diff': [], 'IS % < 0': [],
             'OS Avg Diff': [], 'OS Med Diff': [], 'OS % < 0': []}

    for i, col_factor in enumerate(col_3_factors):
        stats['Model'].append(', '.join(col_factor))

        fs_diff = np.array(fs_sharpe_ratios_diff[i])
        stats['FS Avg Diff'].append(np.mean(fs_diff))
        stats['FS Med Diff'].append(np.median(fs_diff))
        stats['FS % < 0'].append(np.mean(fs_diff < 0) * 100)

        is_diff = np.array(is_sharpe_ratios_diff[i])
        stats['IS Avg Diff'].append(np.mean(is_diff))
        stats['IS Med Diff'].append(np.median(is_diff))
        stats['IS % < 0'].append(np.mean(is_diff < 0) * 100)

        os_diff = np.array(os_sharpe_ratios_diff[i])
        stats['OS Avg Diff'].append(np.mean(os_diff))
        stats['OS Med Diff'].append(np.median(os_diff))
        stats['OS % < 0'].append(np.mean(os_diff < 0) * 100)

    stats_df = pd.DataFrame(stats)

    return stats_df


def squared_sharpe_ratio(sample, factor, weights=None):
    if weights is None:
        mu_f = sample[factor].mean().values.reshape(-1, 1) 
        sigma_f = sample[factor].cov().values
        sigma_f_inv = np.linalg.inv(sigma_f)

        # maximum squared Sharpe ratio
        sh2_f = mu_f.T @ sigma_f_inv @ mu_f
        sh2_f = sh2_f[0,0]

    else:
        weights = weights.flatten()
        returns = (sample[factor] * weights).sum(axis=1)
        mu_f = returns.mean()
        sigma_f = returns.std()

        sh2_f = (mu_f / sigma_f) ** 2

    return sh2_f

def tangency_portfolio_weights(is_sample, factor):
    """
    Calculate the tangency portfolio weights.
    
    :param expected_returns: A vector of expected returns (mu)
    :param covariance_matrix: The covariance matrix (Sigma)
    :param risk_free_rate: The risk-free rate (rf)
    :return: A vector of tangency portfolio weights (t)

    source: https://bookdown.org/compfinezbook/introcompfinr/Efficient-portfolios-of.html - eq. 12.26
    """
    mu_f = is_sample[factor].mean().values.reshape(-1, 1) 
    sigma_f = is_sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    numerator = sigma_f_inv @ (mu_f)
    denominator = np.ones(len(mu_f)) @ numerator
    
    tangency_weights = numerator / denominator
    
    return tangency_weights

In [11]:
N = 100000

summary_df = pd.DataFrame()

col_3_factors = [model.split(', ') for model in top_3_actual['Model'].values]
row_factors = [model.split(', ') for model in table_3_summary['Model'].values]
# print(row_factors)
for row_factor in row_factors:
    stats_df = simulate_sharpe_ratios(master, row_factor, col_3_factors, n_simulations=N)
    stats_df = stats_df.set_index('Model').T

    stats_df['Row_Model'] = ', '.join(row_factor)
    
    stats_df = stats_df.reset_index().melt(id_vars=['index', 'Row_Model'], var_name='Column_Model', value_name='Value')
    stats_df = stats_df.pivot_table(index=['Row_Model', 'Column_Model'], columns='index', values='Value')
    stats_df.columns = [f'{col}' for col in stats_df.columns]

    summary_df = pd.concat([summary_df, stats_df])

summary_df = summary_df.reset_index()

summary_df.to_excel('./result/tableA2.xlsx', index=False)
summary_df

,Row_Model,Column_Model,FS % < 0,FS Avg Diff,FS Med Diff,IS % < 0,IS Avg Diff,IS Med Diff,OS % < 0,OS Avg Diff,OS Med Diff
0,"Mkt, SMB, HML, RMW, CMA","Mkt, S-F, H-F, R-F, C-F",69.552,-0.018996,-0.018798,84.682,-0.019569,-0.018855,85.778,-0.018638,-0.017918
1,"Mkt, SMB, HML, RMW, CMA","Mkt, SMB, HML_s, RMW_s, CMA_s",20.772,0.032836,0.032776,10.056,0.032468,0.031680,9.121,0.032192,0.031267
2,"Mkt, SMB, HML_s, RMW_s, CMA_s","Mkt, S-F, H-F, R-F, C-F",91.136,-0.052144,-0.051846,96.251,-0.051902,-0.050685,96.813,-0.050628,-0.049264
3,"Mkt, SMB, HML_s, RMW_s, CMA_s","Mkt, SMB, HML, RMW, CMA",79.023,-0.032743,-0.032850,90.057,-0.032500,-0.031651,90.817,-0.032177,-0.031179
4,"Mkt, S-F, H-F, R-F, C-F","Mkt, SMB, HML, RMW, CMA",29.923,0.019456,0.019300,15.299,0.019607,0.018808,14.219,0.018506,0.017816
5,"Mkt, S-F, H-F, R-F, C-F","Mkt, SMB, HML_s, RMW_s, CMA_s",8.875,0.052169,0.051853,3.821,0.051931,0.050773,3.174,0.050787,0.049417
6,"Mkt, S-F, H_s-F, R_s-F, C_s-F","Mkt, S-F, H-F, R-F, C-F",39.149,0.009627,0.009417,32.155,0.009958,0.009513,31.815,0.009239,0.008907
7,"Mkt, S-F, H_s-F, R_s-F, C_s-F","Mkt, SMB, HML, RMW, CMA",21.367,0.028691,0.028452,15.623,0.029295,0.028145,14.632,0.027917,0.026927
8,"Mkt, S-F, H_s-F, R_s-F, C_s-F","Mkt, SMB, HML_s, RMW_s, CMA_s",5.146,0.061652,0.061114,0.421,0.062154,0.060307,0.398,0.060085,0.058406
9,"Mkt, S-F, L-F, W-F, A-F","Mkt, S-F, H-F, R-F, C-F",28.742,0.018699,0.018604,28.798,0.019286,0.018340,27.759,0.017809,0.016691


In [13]:
def quote_factors(factors):
    return [f"Q('{factor}')" if '-' in factor else factor for factor in factors]

def spanning_regression(data, factors):
    results = []
    sh2_f = top_3_actual[top_3_actual['Model'] == ', '.join(factors)]['Actual'].iloc[0]  # Calculate Sh^2(f) for the model

    for i, factor in enumerate(factors):
        rhs_factors = [f for j, f in enumerate(factors) if j != i]
        quoted_rhs_factors = quote_factors(rhs_factors)
        quoted_lhs_factor = f"Q('{factor}')" if '-' in factor else factor
        formula = f"{quoted_lhs_factor} ~ {' + '.join(quoted_rhs_factors)}"
        print(f"Running regression: {formula}")
        
        model = smf.ols(formula, data=data).fit()

        intercept = model.params['Intercept'] 
        t_stat = model.tvalues['Intercept']    
        r_squared = model.rsquared
        std_error = model.resid.std()
        
        marginal_contribution = (intercept ** 2) / (std_error ** 2)
        
        result = {
            'LHS': factor,
            'a': intercept,
        }

        for f in rhs_factors:
            param_name = f"Q('{f}')" if '-' in f else f
            result[f] = model.params[param_name]
        
        stat = {
            't(a)': t_stat,
            'R2': r_squared,
            's(e)': std_error,
            'Sh2(f)': sh2_f,
            'a2/s2(e)': marginal_contribution
        }

        result.update(stat)
        
        results.append(result)
    
    results_df = pd.DataFrame(results)
    
    return results_df

In [14]:
factor_a = ['Mkt', 'SMB', 'HML', 'RMW', 'CMA']
factor_b = ['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s']
factor_c = ['Mkt', 'S-F', 'H-F', 'R-F', 'C-F']

t5_panel_a_comb = spanning_regression(master[factor_a], factor_a)
t5_panel_b_small = spanning_regression(master[factor_b], factor_b)
t5_panel_c_small_long = spanning_regression(master[factor_c], factor_c)

Running regression: Mkt ~ SMB + HML + RMW + CMA
Running regression: SMB ~ Mkt + HML + RMW + CMA
Running regression: HML ~ Mkt + SMB + RMW + CMA
Running regression: RMW ~ Mkt + SMB + HML + CMA
Running regression: CMA ~ Mkt + SMB + HML + RMW
Running regression: Mkt ~ SMB + HML_s + RMW_s + CMA_s
Running regression: SMB ~ Mkt + HML_s + RMW_s + CMA_s
Running regression: HML_s ~ Mkt + SMB + RMW_s + CMA_s
Running regression: RMW_s ~ Mkt + SMB + HML_s + CMA_s
Running regression: CMA_s ~ Mkt + SMB + HML_s + RMW_s
Running regression: Mkt ~ Q('S-F') + Q('H-F') + Q('R-F') + Q('C-F')
Running regression: Q('S-F') ~ Mkt + Q('H-F') + Q('R-F') + Q('C-F')
Running regression: Q('H-F') ~ Mkt + Q('S-F') + Q('R-F') + Q('C-F')
Running regression: Q('R-F') ~ Mkt + Q('S-F') + Q('H-F') + Q('C-F')
Running regression: Q('C-F') ~ Mkt + Q('S-F') + Q('H-F') + Q('R-F')


In [15]:
t5_panel_a_comb

,LHS,a,SMB,HML,RMW,CMA,t(a),R2,s(e),Sh2(f),a2/s2(e),Mkt
0,Mkt,0.802725,0.237876,0.089289,-0.385872,-0.937295,5.051890,0.238860,3.871794,0.107304,0.042984,NaN
1,SMB,0.328217,NaN,0.059986,-0.431532,-0.147583,2.845841,0.165780,2.772740,0.107304,0.014012,0.121995
2,HML,-0.027462,0.030863,NaN,0.153428,1.007355,-0.329880,0.493656,1.988845,0.107304,0.000191,0.023560
3,RMW,0.420954,-0.226886,0.156788,NaN,-0.329127,5.103910,0.181928,2.010509,0.107304,0.043839,-0.104047
4,CMA,0.256317,-0.034125,0.452721,-0.144744,NaN,4.671156,0.550913,1.333291,0.107304,0.036958,-0.111148


In [16]:
t5_panel_b_small

,LHS,a,SMB,HML_s,RMW_s,CMA_s,t(a),R2,s(e),Sh2(f),a2/s2(e),Mkt
0,Mkt,0.800059,0.286709,-0.249477,-0.112416,-0.520292,4.861171,0.209385,3.946049,0.140338,0.041107,NaN
1,SMB,0.290506,NaN,-0.145106,-0.270728,0.113976,2.488018,0.172247,2.761973,0.140338,0.011063,0.140461
2,HML_s,-0.056206,-0.077978,NaN,0.530923,1.051308,-0.653676,0.593365,2.024710,0.140338,0.000771,-0.065680
3,RMW_s,0.443073,-0.145245,0.530046,NaN,-0.815954,5.267613,0.409033,2.023037,0.140338,0.047967,-0.029547
4,CMA_s,0.324587,0.025662,0.440480,-0.342436,NaN,5.993650,0.558010,1.310572,0.140338,0.061340,-0.057391


In [17]:
t5_panel_c_small_long

,LHS,a,S-F,H-F,R-F,C-F,t(a),R2,s(e),Sh2(f),a2/s2(e),Mkt
0,Mkt,-0.216716,-0.304819,-0.111947,0.716425,0.632463,-5.564713,0.953094,0.961161,0.08835,0.050838,NaN
1,S-F,-0.352424,NaN,-0.097802,0.823273,1.307000,-5.059859,0.922632,1.712019,0.08835,0.042376,-0.967090
2,H-F,0.002095,-0.072777,NaN,0.327831,0.967777,0.034177,0.913405,1.476829,0.08835,0.000002,-0.264290
3,R-F,0.173620,0.290079,0.155231,NaN,-0.221162,4.173126,0.957495,1.016237,0.08835,0.029188,0.800882
4,C-F,0.154659,0.325507,0.323904,-0.156323,NaN,4.429101,0.970028,0.854379,0.08835,0.032768,0.499741


In [18]:
t5_panel_a_comb.to_excel('./result/tableA3_a.xlsx', index=False)
t5_panel_b_small.to_excel('./result/tableA3_b.xlsx', index=False)
t5_panel_c_small_long.to_excel('./result/tableA3_c.xlsx', index=False)

In [19]:
def tangency_portfolio_weights(is_sample, factor, model_type='spread'):
    """
    Calculate the tangency portfolio weights.
    
    :param expected_returns: A vector of expected returns (mu)
    :param covariance_matrix: The covariance matrix (Sigma)
    :param risk_free_rate: The risk-free rate (rf)
    :return: A vector of tangency portfolio weights (t)

    source: https://bookdown.org/compfinezbook/introcompfinr/Efficient-portfolios-of.html - eq. 12.26
    """
    mu_f = is_sample[factor].mean().values.reshape(-1, 1) 
    sigma_f = is_sample[factor].cov().values
    sigma_f_inv = np.linalg.inv(sigma_f)

    numerator = sigma_f_inv @ (mu_f)
    denominator = np.ones(len(mu_f)) @ numerator

    tangency_weights = numerator / denominator * 100
    
    if model_type == 'spread':
        mkt_weight = tangency_weights[0, 0]
        scaled_weights = (tangency_weights / mkt_weight) * 100
        leverage = np.sum(np.abs(scaled_weights[1:])) / 100  
        scaled_weights = tangency_weights
        leverage = np.sum(np.abs(scaled_weights[scaled_weights < 0])) / 100  #
    
    sh2_f = (mu_f.T @ sigma_f_inv @ mu_f)[0, 0]
    
    return sh2_f, scaled_weights.flatten(), leverage

In [20]:
all_factors = [model.split(', ') for model in table_3_summary['Model'].values]
all_factors

[['Mkt', 'SMB', 'HML', 'RMW', 'CMA'],
 ['Mkt', 'SMB', 'HML_s', 'RMW_s', 'CMA_s'],
 ['Mkt', 'S-F', 'H-F', 'R-F', 'C-F'],
 ['Mkt', 'S-F', 'H_s-F', 'R_s-F', 'C_s-F'],
 ['Mkt', 'S-F', 'L-F', 'W-F', 'A-F'],
 ['Mkt', 'S-F', 'L_s-F', 'W_s-F', 'A_s-F']]

In [21]:
result_df = pd.DataFrame()

for factors in all_factors:
    data = master[factors]  

    model_type = 'spread' if 'SMB' in factors else 'excess'
    sh2_f, weights, leverage = tangency_portfolio_weights(data, factors, model_type=model_type)

    mkt_weight = 100 if model_type == 'spread' else weights[0]
    result = {
        'Model': ', '.join(factors),
        'Sh2(f)': sh2_f,
        'Mkt': mkt_weight, 
        'Size': weights[1],
        'Value': weights[2],
        'Prof': weights[3],
        'Inv': weights[4],
        'Leverage': leverage
    }
    
    result_df = pd.concat([result_df, pd.DataFrame([result])], ignore_index=True)

result_df

,Model,Sh2(f),Mkt,Size,Value,Prof,Inv,Leverage
0,"Mkt, SMB, HML, RMW, CMA",0.107304,100.000000,79.726066,-12.965394,194.482612,269.268524,5.564426
1,"Mkt, SMB, HML_s, RMW_s, CMA_s",0.140338,100.000000,74.117244,-26.684429,210.702800,367.800242,6.793047
2,"Mkt, S-F, H-F, R-F, C-F",0.088350,-897.966961,-460.267528,3.676411,643.534149,811.023930,13.582345
3,"Mkt, S-F, H_s-F, R_s-F, C_s-F",0.079124,-147.377805,-841.550484,281.776018,245.238093,561.914178,9.889283
4,"Mkt, S-F, L-F, W-F, A-F",0.070671,358.084430,246.355620,-21.492826,-218.683392,-264.263833,5.044401
5,"Mkt, S-F, L_s-F, W_s-F, A_s-F",0.078162,121.808013,441.560599,5.414801,-197.043724,-271.739690,4.687834


In [22]:
result_df.to_excel('./result/tableA4.xlsx', index=False)